In [1]:
from fx_list import get_data
from fx_list import correlation
import matplotlib.pyplot as plt
import scipy.stats as sts
import pandas as pd
import os

gym_data = pd.read_csv("data_sources/US_states_gyms_2019.csv")
pactive_data = pd.read_csv("data_sources/physical_activeness_states_2019.csv") 

#merge gym data and physical activeness data, dropping duplicated columns then rename
activeness_df = pd.merge(gym_data, pactive_data, on='alpha code', how='right')
activeness_df = activeness_df.drop(columns=['state_y', 'state code_y'])
activeness_df.rename(columns ={'state_x': 'State','state code_x': 'State Code'}, inplace = True)
activeness_df['State'] = activeness_df['State'].str.title() #capitalise State names so can be merged with obesity_data

get_data('Obesity')

obesity_data = pd.read_csv("data_sources/obtained_data_Obesity.csv")
obesity_data
#checking with obesity_data before renaming and dropping unwanted rows and columns 
obesity_data.rename(columns={'State Name':'State', 'Value': 'Obesity'}, inplace = True) # rename column NAME to State to match up with activeness_df 
obesity_data= obesity_data.drop([50,51]) #dropping index number for District of Columbia and United States

#merge activeness_df with obesity_data
obesevactive_df = pd.merge(activeness_df, obesity_data, how='outer')
obesevactive_df.reset_index(drop=True, inplace=True)

#output csv file
obesevactive_df.to_csv('data_sources\obese_vs_activeness.csv',index = False, header=True)
obesevactive_df


,State,State Code,alpha code,total gyms,per 100K residents (%),prevalance of inactivity,meets cdc requirement (%),adolescents physically active (%),Edition,Report Type,Measure Name,Rank,Obesity,Score,Lower CI,Upper CI,Source,Source Year
0,Alabama,1,AL,385,7.9,31.0,17.6,23.2,2019,2019 Annual,Obesity,45.0,36.2,1.38,37.9,34.6,"CDC, Behavioral Risk Factor Surveillance System",2018
1,Alaska,2,AK,90,12.2,20.3,25.6,17.9,2019,2019 Annual,Obesity,15.0,29.5,-0.37,32.1,26.9,"CDC, Behavioral Risk Factor Surveillance System",2018
2,Arizona,4,AZ,318,4.5,23.7,25.5,22.0,2019,2019 Annual,Obesity,15.0,29.5,-0.37,31.2,27.7,"CDC, Behavioral Risk Factor Surveillance System",2018
3,Arkansas,5,AR,134,4.4,32.5,19.3,22.7,2019,2019 Annual,Obesity,48.0,37.1,1.62,39.1,35.1,"CDC, Behavioral Risk Factor Surveillance System",2018
4,California,6,CA,3365,8.5,20.4,22.6,20.5,2019,2019 Annual,Obesity,5.0,25.8,-1.33,26.9,24.8,"CDC, Behavioral Risk Factor Surveillance System",2018
5,Colorado,8,CO,551,9.7,17.3,27.4,25.4,2019,2019 Annual,Obesity,1.0,22.9,-2.00,24.0,21.9,"CDC, Behavioral Risk Factor Surveillance System",2018
6,Connecticut,9,CT,575,16.0,22.7,26.1,23.2,2019,2019 Annual,Obesity,7.0,27.4,-0.91,28.6,26.2,"CDC, Behavioral Risk Factor Surveillance System",2018
7,Delaware,10,DE,142,14.6,28.4,23.8,NaN,2019,2019 Annual,Obesity,33.0,33.5,0.68,35.3,31.7,"CDC, Behavioral Risk Factor Surveillance System",2018
8,Florida,12,FL,2463,11.6,28.0,27.0,22.7,2019,2019 Annual,Obesity,24.0,30.7,-0.05,32.2,29.1,"CDC, Behavioral Risk Factor Surveillance System",2018
9,Georgia,13,GA,115,9.6,28.5,24.1,24.0,2019,2019 Annual,Obesity,30.0,32.5,0.42,33.7,31.2,"CDC, Behavioral Risk Factor Surveillance System",2018


In [ ]:
size = [2*n for n in range(len(obesevactive_df['Obesity']))]
plt.scatter(obesevactive_df['Obesity'], obesevactive_df['per 100K residents (%)'], color='yellow', s = size, edgecolors="red")
plt.title(f'Obesity rate vs. Gyms per 100K residents')
plt.xlabel('Obesity Rate (%)')
plt.ylabel('Gyms per 100K residents (%)')

#call correlation function to get the linear correlation for Obesity and Per 100K residents
correlation(obesevactive_df['Obesity'], obesevactive_df['per 100K residents (%)'], 30, 30)

#to print out names on plots
for n, txt in enumerate(obesevactive_df['State']):
    plt.annotate(txt, (obesevactive_df['Obesity'][n], obesevactive_df['per 100K residents (%)'][n]))
#the above can be commented out if scatterplot looks junky. enable just to have a look.

plt.savefig("output/obese_vs_gyms.png")

The relationship between the Obesity Rate of a state and available Gyms per 100K state residents is a low negative (weak) correlation with the R-value of -0.34. This means despite having more gyms per 100K state residents, it has not much affect on the obesity rates in a state. An interesting observation that can support this is the data plot which has the highest % of gyms per 100K residents (Washington) still have a higher rate of Obesity compared to those with less gyms available (Colorado, Hawaii)

In [ ]:
#let's try to see if the CDC is right on physical activeness by identifying the relationship between activeness and obesity 
size = [2*n for n in range(len(obesevactive_df['Obesity']))]
ax = plt.gca()
ax.scatter(obesevactive_df['Obesity'], obesevactive_df['meets cdc requirement (%)'], color='blue', s = size, edgecolors="gray")
ax.scatter(obesevactive_df['Obesity'], obesevactive_df['prevalance of inactivity'], color='orange', s = size)
plt.title(f'Obesity rate vs. Activeness & Inactivity')
plt.xlabel('Obesity Rate (%)')
plt.ylabel('Activeness(%)')

#dropping State: New Jersey as no data is available for Activeness
no_newjersey= obesevactive_df.drop([29])

#call correlation function to get the linear correlation for Obesity and Activeness
correlation(no_newjersey['Obesity'], no_newjersey['meets cdc requirement (%)'], 28, 17.5)
correlation(obesevactive_df['Obesity'], obesevactive_df['prevalance of inactivity'], 29, 27)
plt.show()
plt.savefig("output/obese_vs_active.png")

As one of the causes of Obesity defined by the CDC is due to physical inactivity, there is a strong (but not perfect) negative relationship between Obesity rate and physical activeness (R2= -0.73) while a positive strong relationship between Obesity rate and physical inactivity (R2 = 0.73). This proves that the state residents that meets the CDC's physical activeness requirements have a lower obesity rate. However as this has become almost common knowledge, there is nothing to explore here.

In [ ]:
#gyms vs activeness
plt.scatter(obesevactive_df['prevalance of inactivity'], obesevactive_df['total gyms'], color='yellow', edgecolors="red")
plt.title(f'Obesity rate vs. Gyms per 100K residents')
plt.xlabel('Obesity Rate (%)')
plt.ylabel('Gyms per 100K residents (%)')
correlation(obesevactive_df['prevalance of inactivity'], obesevactive_df['total gyms'], 30, 30)